In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [47]:
%pylab inline
from ipywidgets import interact
import seaborn as sns
sns.set_context('poster')

Populating the interactive namespace from numpy and matplotlib


### FPGA and Spiketag Configuration

In [48]:
from spiketag.base import ProbeFactory, bload
from spiketag.mvc  import Sorter
from spiketag.fpga import xike_config

#### spiketag

In [49]:
nCh = 160
fs = 25000.
tetrodes = ProbeFactory.genTetrodeProbe(fs=fs, n_ch=nCh)
tetrodes.fromfile('./open-ephys-load/40 tetrode_channel_map')

In [50]:
tetrodes.ch_hash(14)

array([ 0,  1, 14, 15])

In [51]:
tetrodes.reorder_by_chip = True
tetrodes._nchips = 5

#### FPGA

In [52]:
config = xike_config(probe=tetrodes, offset_value=32, thres_value=-500)

In [53]:
config.ch_ugp[14]

(0, 1, 14, 15)

### Sorting

In [56]:
!cp spk.bin mua.bin.spk

In [57]:
%gui qt

In [58]:
sorter = Sorter('/disk0/testbench/mua.bin', 
                probe=tetrodes, 
                fet_method='pca', 
                clu_method='hdbscan',
                n_jobs=24)

2017-08-16 17:13:43,998 - spiketag - INFO - load mua data
2017-08-16 17:13:44,001 - spiketag - INFO - #############  load data  ###################
2017-08-16 17:13:44,002 - spiketag - INFO - /disk0/testbench/mua.bin loaded, it contains: 
2017-08-16 17:13:44,003 - spiketag - INFO - 166017 * 160 points (106250880 bytes) 
2017-08-16 17:13:44,005 - spiketag - INFO - 160 channels with sampling rate of 25000.0000 
2017-08-16 17:13:44,006 - spiketag - INFO - 6.641 secs (0.111 mins) of data
2017-08-16 17:13:44,007 - spiketag - INFO - #############################################
2017-08-16 17:13:44,109 - spiketag - INFO - reordered with nchips=5 and nch_perchip=32
2017-08-16 17:13:44,175 - spiketag - INFO - raw data have 7303 spks
2017-08-16 17:13:44,177 - spiketag - INFO - removing all spks on group which len(spks) less then fetlen
2017-08-16 17:13:44,182 - spiketag - INFO - removed all spks on these groups: {32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 18: 0, 19: 0, 22: 0, 23: 0, 24: 0, 25: 0,

In [59]:
spks = np.fromfile('./spk.bin', dtype=np.int32).reshape(-1,2).T
spks

array([[     0,      5,      5, ..., 165933, 165933, 165991],
       [    82,     56,     58, ...,     57,     59,     68]], dtype=int32)

In [60]:
sorter.show_mua(chs=tetrodes[0], spks=spks)

### Calculate and Download Threshold

In [30]:
thr = sorter.model.mua.get_threshold()

In [31]:
thr

array([-283.14520264, -276.16522217, -310.15524292, -274.3416748 ,
       -267.74752808, -260.79141235, -269.56164551, -274.4654541 ,
       -313.02194214, -281.74084473, -304.44064331, -277.62969971,
       -300.39178467, -302.0994873 , -296.72589111, -299.1227417 ,
       -327.89624023, -323.41088867, -345.29556274, -326.10961914,
       -297.66189575, -291.78442383, -292.42437744, -296.15325928,
       -310.00323486, -323.5838623 , -315.53173828, -308.92962646,
       -280.25317383, -279.90353394, -277.38574219, -270.51071167,
       -264.37841797, -266.21356201, -265.58010864, -263.95275879,
       -285.34954834, -275.87420654, -302.82559204, -278.24502563,
       -303.13760376, -291.53253174, -300.64227295, -297.08636475,
       -301.0201416 , -293.93371582, -286.50708008, -296.45080566,
       -350.64385986, -354.0824585 , -350.59173584, -350.5171814 ,
       -377.69226074, -379.65911865, -388.90280151, -385.05953979,
       -378.18450928, -385.53878784, -374.17401123, -379.42022

In [22]:
config.thres[0:84] = thr[:84]

In [23]:
config.thres

threshold of ch0 is -301.710034053
threshold of ch1 is -297.682166188
threshold of ch2 is -333.306789752
threshold of ch3 is -299.273333256
threshold of ch4 is -282.757192828
threshold of ch5 is -279.909307821
threshold of ch6 is -289.660454503
threshold of ch7 is -291.651947044
threshold of ch8 is -323.043834553
threshold of ch9 is -288.898895015
threshold of ch10 is -317.061393857
threshold of ch11 is -291.574488047
threshold of ch12 is -319.883941809
threshold of ch13 is -311.185368792
threshold of ch14 is -310.677903771
threshold of ch15 is -310.302915354
threshold of ch16 is -343.925911555
threshold of ch17 is -335.186075334
threshold of ch18 is -359.456802493
threshold of ch19 is -339.550564075
threshold of ch20 is -302.049550593
threshold of ch21 is -300.307809025
threshold of ch22 is -301.497926705
threshold of ch23 is -305.166008386
threshold of ch24 is -319.215767467
threshold of ch25 is -321.264449361
threshold of ch26 is -319.481444589
threshold of ch27 is -314.788298045
th

threshold enable status: True

In [45]:
!rm *.bin

### Go Sorting Again

In [46]:
!cp spk.bin mua.bin.spk

cp: cannot stat 'spk.bin': No such file or directory


In [16]:
sorter = Sorter('/disk0/testbench/mua.bin', 
                probe=tetrodes, 
                fet_method='pca', 
                clu_method='hdbscan',
                n_jobs=24)

2017-08-16 16:57:35,644 - spiketag - INFO - load mua data
2017-08-16 16:57:35,647 - spiketag - INFO - #############  load data  ###################
2017-08-16 16:57:35,648 - spiketag - INFO - /disk0/testbench/mua.bin loaded, it contains: 
2017-08-16 16:57:35,649 - spiketag - INFO - 613492 * 160 points (392634880 bytes) 
2017-08-16 16:57:35,650 - spiketag - INFO - 160 channels with sampling rate of 25000.0000 
2017-08-16 16:57:35,651 - spiketag - INFO - 24.540 secs (0.409 mins) of data
2017-08-16 16:57:35,653 - spiketag - INFO - #############################################
2017-08-16 16:57:35,929 - spiketag - INFO - reordered with nchips=5 and nch_perchip=32
2017-08-16 16:57:36,108 - spiketag - INFO - raw data have 26152 spks
2017-08-16 16:57:36,109 - spiketag - INFO - removing all spks on group which len(spks) less then fetlen
2017-08-16 16:57:36,116 - spiketag - INFO - removed all spks on these groups: {32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 18: 0, 19: 0, 22: 0, 23: 0, 24: 0, 25: 

In [17]:
sorter.run()